In [3]:
%pip install -q sql av pillow pandas sqlalchemy ipython-sql pymysql roboflow git+https://github.com/THU-MIG/yolov10.git supervision huggingface_hub bottleneck==1.3.6 numexpr==2.8.4 wget

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [96]:
import pandas as pd
import pymysql
from pymysql.err import IntegrityError, OperationalError
import os
from PIL import Image
import numpy as np
import supervision as sv
import cv2
import av
from PIL import Image
import shutil as sh
import re
from roboflow import Roboflow
import cv2
from ultralytics import YOLOv10
import wget

model_path = '../Jupyter_local_code/best.pt'


In [110]:
model = YOLOv10(model_path)
bba = sv.BoxCornerAnnotator()
la = sv.LabelAnnotator(text_scale = 0.4, text_padding = 1)

def annotate_image(input_img, model, label_annotator = la, bounding_box_annotator = bba, verbose = False, conf = True, conf_level = 0.05):
    
    cont_img = np.ascontiguousarray(input_img, dtype=np.uint8)
    results = model(cont_img, conf=conf_level, verbose = verbose)[0]
    conf_array = np.array(results.boxes.conf.cpu())
    conf_ls_str  = [str(round(x * 100) ) + '%' for x in conf_array]
    tot_time = sum([x for x in results.speed.values()])
    
    detections = sv.Detections.from_ultralytics(results)
    annotated_image = bounding_box_annotator.annotate(
        scene=input_img, detections=detections)
    num_oysters = detections.xyxy.shape[0]
    annotated_image = label_annotator.annotate(
        scene=annotated_image, detections=detections, labels = conf_ls_str)
    return(annotated_image, num_oysters, tot_time)


def annotate_video(input_vid, out_path = '.', model_file = 'best.pt', im_width = 416, im_height = 416):
    bba = sv.BoundingBoxAnnotator()
    la = sv.LabelAnnotator()
    model = YOLOv10(model_file)

    container = av.open(input_vid)
    stream_vid = container.streams.video[0]
    fname = input_vid.rsplit('/', 1)[-1]
    per_index = fname.index('.')
    out_path = f'{out_path}/{fname[:per_index]}_annotated.mp4'
    outp = av.open(out_path, 'w')
    codec_name = stream_vid.codec_context.name
    fps = stream_vid.codec_context.rate
    output_stream = outp.add_stream(codec_name, str(fps))
    output_stream.width = im_width
    output_stream.height = im_height
    output_stream.pix_fmt = stream_vid.codec_context.pix_fmt
    for index, frame in enumerate(container.decode(stream_vid)):
        pil_img = frame.to_image()
        np_img = np.array(pil_img)
        np_img_resize = cv2.resize(np_img, (im_width, im_height))
        np_rot = np_img_resize[:, :, ::-1]
        small_pil_img = Image.fromarray(np_rot)
        np_image_2 = np.array(small_pil_img)
        an_mg, _, _2 = annotate_image(np_image_2, label_annotator = la, bounding_box_annotator=bba, model_pred = model, verbose = False)
        frame_out = av.VideoFrame.from_ndarray(an_mg, format='bgr24')
        pkt = output_stream.encode(frame_out)
        outp.mux(pkt)
    container.close()
    outp.close()

    print(f'Annotated video has been saved as {out_path}')
    return True

In [98]:
db_main = 'test_4'
cur_name = 'ab'


In [99]:
sql_conn = pymysql.connect(
    user="root",
    password="dbuserdbuser",
    host="localhost",
    port=3306,
    database=db_main,
    cursorclass=pymysql.cursors.DictCursor,
    autocommit=True)

cur = sql_conn.cursor()
res = cur.execute("SELECT 1")
res = cur.fetchall()
res

[{'1': 1}]

In [100]:
def add_user(name, password):
    '''
    Returns: 1 if query is successful, 0 if not
    '''

    try:
        res = cur.execute(f"INSERT INTO people (Username, Password, Time_Created) VALUES ('{name}', '{password}', CURRENT_TIMESTAMP );")
    except IntegrityError:
        print(f"Data Integrity Violated: Duplicate User with {name}")
        res = 0
    return res
add_user(cur_name, '123')

Data Integrity Violated: Duplicate User with ab


0

In [101]:
def get_ext(fpath):
    fname = fpath.rsplit('/', 1)[-1]
    per_index = fname.index('.')
    ext = fname[per_index + 1:].lower()
    return ext

def get_id_fname(f_out, fpath, id):
    fname = fpath.rsplit('/', 1)[-1]
    per_index = fname.index('.')
    ext = fname[per_index + 1:].lower()
    return f"{f_out}/{fname[:per_index]}-{id}.{ext}"

def get_REPLACE_ID(column_id = 'Raw_File_ID', table='raw_files', column_rep='Filepath'):
    cur.execute(f"""SELECT {column_id} from {table} where {column_rep} = 'REPLACE'""")
    id = cur.fetchall()[-1][column_id]
    return id

def get_match(pattern, string):
    matches = re.search(pattern, string)
    if matches:

        print(matches.group(1))
        return(matches.group(1))
    else:
        raise ValueError(f'Improper string exported, couldn\'t find {pattern} in the given text')

def delete_folder(name, base = "."):
    fpath = os.path.join(base, name)
    if os.path.exists(fpath):
        for item in os.listdir(fpath):
            new_path = os.path.join(fpath, item)
            if os.path.isfile(new_path):
                os.remove(new_path)
            elif os.path.isdir(new_path):
                delete_folder(item, fpath)
        os.rmdir(fpath)

def make_folder(fpath, overwrite = False):
    delete_folder(fpath, '.') if overwrite else None
    if os.path.exists(fpath):
        
        print(f"{fpath} already exists")
    else:
        os.mkdir(fpath)

In [102]:
make_folder('Files')
make_folder('Files/Video_raw')
make_folder('Files/Video_ann')
make_folder('Files/Image_raw')
make_folder('Files/Image_ann')
make_folder('Files/Model')


def add_photo(name, fpath, notes = '', f_out = './Files/Image_raw'):
    '''
    Returns: Index of added photo if successful, 0 if not
    '''

    im = Image.open(fpath)
    width = im.size[0]
    height = im.size[1]
    fsize = os.stat(fpath).st_size
    ext = get_ext(fpath)

    cur.execute(f"INSERT INTO raw_files (Username, Filepath, Size, Type, Extension, Notes, Width, Height, Time_Uploaded) VALUES ('{name}', 'REPLACE', {fsize}, 'Image', '{ext}', '{notes}', {width}, {height}, CURRENT_TIMESTAMP);")
    id = get_REPLACE_ID()
    f_id_name = get_id_fname(f_out, fpath, id)
    im.save(f_id_name)
    cur.execute(f"UPDATE raw_files SET Filepath = '{f_id_name}' WHERE Raw_File_ID = {id};")
    return id

SOME_IMAGE_PATH = "/mnt/linuxlab/home/mstaus1/Desktop/Shellfish_project_2024/Jupyter_local_code/problem_photo.jpg"



id_raw_photo = add_photo(cur_name, fpath = SOME_IMAGE_PATH, notes = 'from HPC')

Files already exists
Files/Video_raw already exists
Files/Video_ann already exists
Files/Image_raw already exists
Files/Image_ann already exists
Files/Model already exists


In [109]:
import cv2
def add_video(name, fpath, notes = '', f_out = './Files/Video_raw'):
    '''
    Returns: Index of added video if successful, 0 if not
    '''
    cap = cv2.VideoCapture(fpath)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fsize = os.stat(fpath).st_size
    ext = get_ext(fpath)
    
    cur.execute(f"INSERT INTO raw_files (Username, Filepath, Size, Type, Extension, Notes, Width, Height, Time_Uploaded) VALUES ('{name}', 'REPLACE', {fsize}, 'Video', '{ext}', '{notes}', {width}, {height}, CURRENT_TIMESTAMP);")
    id = get_REPLACE_ID()
    f_id_name = get_id_fname(f_out, fpath, id)
    sh.copyfile(fpath, f_id_name)

    cur.execute(f"UPDATE raw_files SET Filepath = '{f_id_name}' WHERE Raw_File_ID = {id};")

    fps = cap.get(cv2.CAP_PROP_FPS)
    color_order = 'RGB' # FIXME - cant figure out how to extract from cv2 object

    cur.execute(f"INSERT INTO videos (Raw_File_ID, FPS, Color_Order) VALUES ('{id}', '{fps}', '{color_order}');")

    return id

SOME_VIDEO_PATH = "/mnt/linuxlab/home/mstaus1/Desktop/Shellfish_project_2024/Jupyter_local_code/GOPR1077.MP4"
id_raw_video = add_video(cur_name, SOME_VIDEO_PATH, notes='on new computer')

In [47]:
make_folder('Files/Roboflow')
        

def add_roboflow(name, export_string, f_out = './Files/Roboflow', f_weights = "./Files/Weights"):
    
    '''
    Returns: Index of added roboflow if successful, 0 if not
    '''
    pattern_api = r'api_key\s*=\s*"([^"]+)"'
    api_key_lab = get_match(pattern_api, export_string)

    pattern_workspace = r'rf\.workspace\("([^"]+)"\)'
    workspace_lab = get_match(pattern_workspace, export_string)

    pattern_project = r'project\("([^"]+)"\)'
    project_lab = get_match(pattern_project, export_string)

    pattern_version = r'version\((\d+)\)'
    version_lab = get_match(pattern_version, export_string)

    pattern_download = r'\bdownload\("([^"]+)"\)'
    download_lab = get_match(pattern_download, export_string)

    folder_roboflow = f"{f_out}/{workspace_lab}_{project_lab}_{version_lab}_{download_lab}"

    # if not os.path.exists(folder_roboflow):
    rf = Roboflow(api_key = api_key_lab)
    project = rf.workspace(workspace_lab).project(project_lab)
    version = project.version(version_lab)
    dat = version.download(download_lab, location = folder_roboflow)
    
    fpath = os.path.join(folder_roboflow, 'data.yaml')

    with open(fpath) as f1:
        lines = f1.readlines()
    with open(fpath, 'w') as f2:
        f2.writelines(lines[:-4])
        f2.write("test: ../test/images\ntrain: ../train/images\nval: ../valid/images\n\n")

    make_folder(f_weights)
    prefix = "https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10"
    versions = ['n', 's', 'm', 'b', 'x', 'l']
    suffix = ".pt"
    for ver in versions:
        web_path = prefix + ver + suffix
        computer_path = os.path.join(f_weights, 'yolov10' + ver + suffix)
        if not os.path.exists(computer_path):
            wget.download(web_path, out = f_weights)
    cur.execute(f"INSERT INTO roboflow (Api_Key, Workspace, Project, Version, Download, Dataset_Location, Username, Timestamp) VALUES ('REPLACE', '{workspace_lab}', '{project_lab}', '{version_lab}', '{download_lab}', '{folder_roboflow}', '{name}', CURRENT_TIMESTAMP);")
    
    id = get_REPLACE_ID('Roboflow_ID', 'roboflow', 'Api_Key')
    
    cur.execute(f"UPDATE roboflow SET Api_Key = '{api_key_lab}' WHERE Roboflow_ID = {id};")
    
    
    
    return id




    
id_rob = add_roboflow(cur_name, \
"""
rf = Roboflow(api_key="Rvk9Clz4ote58KBvCfU3")
project = rf.workspace("curvy-oysters").project("oysters-cjt7n")
dataset = project.version(19).download("yolov8")

""" )




Files/Roboflow already exists
Rvk9Clz4ote58KBvCfU3
curvy-oysters
oysters-cjt7n
19
yolov8
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.34, to fix: `pip install ultralytics==8.0.196`
./Files/Weights already exists


In [104]:
def add_model(roboflow_ID, f_out = "./Files/Model", weights_path = "./Files/Weights/yolov10n.pt", epochs = 10, batch = 32):
#     make_folder(tmp_path)
    cur.execute(f"SELECT * FROM roboflow WHERE Roboflow_ID = {roboflow_ID}")
    res = cur.fetchall()[-1]
    pt = os.path.join(os.getcwd(), res['Dataset_Location'][len('.\\'):], 'data.yaml')
    samp_photo = os.path.join(res['Dataset_Location'], 'test/images')
    pt_rep = pt.replace('\\', '/')
    !yolo task=detect mode=train epochs={epochs} batch={batch} plots=False model={weights_path} data={pt_rep}
    
    
    first_photo = os.listdir(samp_photo)[0]
    im = Image.open(os.path.join(samp_photo, first_photo))
    width = im.size[0]
    height = im.size[1]
                                                               
    cur.execute(f"""INSERT INTO models (Timestamp_Created, Model_Points_Path, Version, 
                Hyperparams, Model_Type, Width_Training_Images, Height_Training_Images, Roboflow_ID) values
                (CURRENT_TIMESTAMP, 'REPLACE', 10, 'epochs = {epochs} batch = {batch}', 'YOLO', {width}, {height}, {roboflow_ID})
                """)
    
    id_mod = get_REPLACE_ID(column_id = 'Model_ID', table='models', column_rep='Model_Points_Path')
    model_path = f'./Files/Model/{id_mod}.pt'
    os.rename('./runs/detect/train/weights/best.pt', model_path)
    
    cur.execute(f"UPDATE models SET Model_Points_Path = '{model_path}' WHERE Model_ID = {id_mod};")

    delete_folder('runs')
#     delete_folder(tmp_path)
    return id_mod

id_mod = add_model(id_rob,epochs=10)
    

New https://pypi.org/project/ultralytics/8.2.40 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.34 🚀 Python-3.9.7 torch-1.10.1+cu113 CUDA:0 (NVIDIA GeForce RTX 4080, 16071MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=./Files/Weights/yolov10n.pt, data=/mnt/linuxlab/home/mstaus1/Desktop/Shellfish_project_2024/sql/Files/Roboflow/curvy-oysters_oysters-cjt7n_19_yolov8/data.yaml, epochs=10, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, val_period=1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det

                 Class     Images  Instances      Box(P          R      mAP50  m
                   all         92       1501      0.331       0.37      0.265      0.122

      Epoch    GPU_mem     box_om     cls_om     dfl_om     box_oo     cls_oo     dfl_oo  Instances       Size
       4/10      6.15G      1.646       1.58      1.652      1.616      1.968   
                 Class     Images  Instances      Box(P          R      mAP50  m/mnt/linuxlab/home/mstaus1/anaconda3/lib/python3.9/site-packages/ultralytics/utils/ops.py:862: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  index = index // nc
                 Class     Images  Instances      Box(P         

In [112]:

def ann_img(Raw_File_ID, Model_ID, notes = '', f_out = './Files/Image_ann'):
    try:
        cur.execute(f"SELECT * FROM raw_files WHERE Raw_File_ID = {Raw_File_ID}")
        cur_photo = cur.fetchall()[-1]

        cur.execute(f"SELECT * FROM models WHERE Model_ID = {Model_ID}")
        cur_model = cur.fetchall()[-1]

        #     print(cur_photo, cur_model)


        im = Image.open(cur_photo['Filepath'])
        np_img = np.array(im)
        model = YOLOv10(cur_model['Model_Points_Path'])
#         print(model)
        annot, num_oysters, tot_time = annotate_image(np_img, model)

        cur.execute(f"INSERT INTO annotated_files (Raw_File_ID, Model_ID, Annotated_Filepath, Time_to_Annotate, Notes, Timestamp) VALUES ('{Raw_File_ID}', '{Model_ID}', 'REPLACE', '{tot_time}', '{notes}', CURRENT_TIMESTAMP);")
        id = get_REPLACE_ID(column_id = 'Ann_File_ID', table='annotated_files', column_rep='Annotated_Filepath')
        f_id_name = get_id_fname(f_out, cur_photo['Filepath'], id)
        print(f_id_name)
        im_f = Image.fromarray(annot)
        im_f.save(f_id_name)
        cur.execute(f"UPDATE annotated_files SET Annotated_Filepath = '{f_id_name}' WHERE Ann_File_ID = {id};")
        cur.execute(f"INSERT INTO annotated_photos (Ann_File_ID, Number_of_Oysters) VALUES ({id}, {num_oysters})")
        
        return id
    except IntegrityError:
        cur.execute(f"SELECT Ann_File_ID FROM annotated_files WHERE Raw_File_ID = {Raw_File_ID} AND Model_ID = {Model_ID}")
        return(cur.fetchall()[-1]['Ann_File_ID'])
        

ann_photo_id = ann_img(id_raw_photo, id_mod)

./Files/Image_ann/problem_photo-9-14.jpg


In [ ]:
from time import time
def annotate_video(input_vid, out_path = '.', model_file = 'best.pt', im_width = 416, im_height = 416):
    tot_oysters = 0
    tot_frame = 0
    
    
    bba = sv.BoundingBoxAnnotator()
    la = sv.LabelAnnotator()
    model = YOLOv10(model_file)

    container = av.open(input_vid)
    stream_vid = container.streams.video[0]
    fname = input_vid.rsplit('/', 1)[-1]
    per_index = fname.index('.')
    out_path = os.path.join(out_path, f'{fname[:per_index]}_annotated.mp4')
    outp = av.open(out_path, 'w')
    codec_name = stream_vid.codec_context.name
    fps = stream_vid.codec_context.rate
    output_stream = outp.add_stream(codec_name, str(fps))
    output_stream.width = im_width
    output_stream.height = im_height
    output_stream.pix_fmt = stream_vid.codec_context.pix_fmt
    start = time()
    for index, frame in enumerate(container.decode(stream_vid)):
        pil_img = frame.to_image()
        np_img = np.array(pil_img)
        np_img_resize = cv2.resize(np_img, (im_width, im_height))
        np_rot = np_img_resize[:, :, ::-1]
        small_pil_img = Image.fromarray(np_rot)
        np_image_2 = np.array(small_pil_img)
        an_mg, num_oysters, _ = annotate_image(np_image_2, model)
        tot_oysters += num_oysters
        frame_out = av.VideoFrame.from_ndarray(an_mg, format='bgr24')
        pkt = output_stream.encode(frame_out)
        outp.mux(pkt)
    end = time()
    container.close()
    outp.close()

    print(f'Annotated video has been saved as {out_path}')
    return tot_oysters / index, end - start

In [ ]:
def ann_video(Raw_File_ID, Model_ID, notes = '', f_out = './Files/Video_ann'):
    try:
        cur.execute(f"SELECT * FROM raw_files WHERE Raw_File_ID = {Raw_File_ID}")
        cur_photo = cur.fetchall()[-1]

        cur.execute(f"SELECT * FROM models WHERE Model_ID = {Model_ID}")
        cur_model = cur.fetchall()[-1]
        
        avg_oysters, annotate_video()
        
        
    except IntegrityError:
        cur.execute(f"SELECT Ann_File_ID FROM annotated_files WHERE Raw_File_ID = {Raw_File_ID} AND Model_ID = {Model_ID}")
        return(cur.fetchall()[-1]['Ann_File_ID'])



In [44]:
# make_folder("./Files/tmp")
# print(os.listdir('.'))
# os.rename('./runs/detect/train/weights/best.pt', './Files/Model/f1/')


./Files/tmp already exists
['yolov8n.pt', 'base_sql.ipynb', 'runs', 'oyster_project.ddl', '.ipynb_checkpoints', 'Files']


In [37]:
print(id_raw_photo)
print(id_raw_video)
print(id_rob)

6
7
17


In [45]:
delete_folder('runs')

In [46]:
delete_folder('tmp')